### Chroma DB

In [1]:
import chromadb

chroma_client = chromadb.Client()

In [3]:
# create a collection
collection = chroma_client.create_collection(name="first_collection")

In [4]:
collection

Collection(name=first_collection)

In [5]:
import pandas as pd
from tqdm import tqdm

df = pd.read_csv('../data/naver-news-summarization-ko/test_10row.csv', encoding='cp949')
df.head(5)

,date,category,press,title,document,link,summary
0,2022.7.4 8:47,economy,머니투데이,아이트로닉스 차량용 복합기능형 졸음 방지 단말기 특허 출원,아이엘사이언스의 자회사 아이트로닉스는 차량용 복합기능형 졸음 방지 단말기 특허를 출...,https://n.news.naver.com/mnews/article/008/000...,"아이일, 아이트로닉스는 차량용 복합기능형 졸음 방지 단말기 특허를 출원했다고 4일 ..."
1,2022.7.7 15:01,IT과학,파이낸셜뉴스,뉴스페이스로 가는 한국… 우주기업 성장할 기술·터전 만든다,한국형 우주발사체 누리호 가 지난 6월 21일 전남 고흥 나로우주센터 발사대에서 날...,https://n.news.naver.com/mnews/article/014/000...,한국형 우주발사체 누리호가 지난 6월 21일 전남 고흥 나로우주센터 발사대에서 성공...
2,2022.7.1 6:08,economy,매일경제,SP500 올 상반기 21%↓…50년래 최악 월가월부,닥터둠 루비니 주가 50% 폭락할 것 뉴욕 증시 3대지수 모두 하락 대형주 중심의 ...,https://n.news.naver.com/mnews/article/009/000...,대 대형주 중심의 미국 스탠더드앤드푸어스 S P 500 지수가 올 상반기 50여년만...
3,2022.7.5 12:29,economy,파이낸셜뉴스,풀무원 알래스칸 명태살 ‘볼카츠’ 출시,파이낸셜뉴스 풀무원식품은 알래스카 청정해역의 신선한 명태살로 만든 연육을 바삭하게 ...,https://n.news.naver.com/mnews/article/014/000...,5일 5일 풀무원식품은 알래스카 청정해역의 신선한 명태살로 만든 연육을 동그랗게 빚...
4,2022.7.3 10:49,IT과학,디지털타임스,일동제약 생산본부장에 강덕원 부사장 영입,강덕원 일동제약 생산본부장. 일동제약 제공 일동제약은 생산본부장으로 강덕원 부사장을...,https://n.news.naver.com/mnews/article/029/000...,일동제약은 생산본부장으로 강덕원 부사장을 영입했다고 지난 1일 밝혔으며 강덕원 본부...


In [6]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('snunlp/KR-SBERT-V40K-KlueNLI-augSTS')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
ids = []
metadatas = []
embeddings = []

for row in tqdm(df.iterrows()):
    index = row[0]
    query = row[1].title
    answer = row[1].summary

    metadata = {
        "query": query,
        "answer": answer
    }

    embedding = model.encode(query, normalize_embeddings=True)

    ids.append(str(index))
    metadatas.append(metadata)
    embeddings.append(embedding)

chunk_size = 1024
total_chunks = len(embeddings) // chunk_size + 1
embeddings = [e.tolist() for e in tqdm(embeddings)]

for chunk_idx in tqdm(range(total_chunks)):
    start_idx = chunk_idx * chunk_size
    end_idx = (chunk_idx + 1) * chunk_size

    chunk_embeddings = embeddings[start_idx: end_idx]
    chunk_ids = ids[start_idx: end_idx]
    chunk_metadatas = metadatas[start_idx: end_idx]

    collection.add(embeddings=chunk_embeddings, ids=chunk_ids, metadatas=chunk_metadatas)

15it [00:00, 229.28it/s]
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 145.03it/s]


In [18]:
result = collection.query(
    query_embeddings=model.encode("코오롱 수소 밸류체인 플랫폼", normalize_embeddings=True).tolist(),
    n_results=3
)

print(result)

{'ids': [['7', '10', '4']], 'embeddings': None, 'documents': [[None, None, None]], 'uris': None, 'data': None, 'metadatas': [[{'answer': '코오롱그룹은 6일 동대문 디자인플라자에서 열린 ‘코리아 H2 비즈니스 서밋 2022 인베스터데이’에 참가해 그룹이 보유한 첨단기술을 바탕으로 수소 생산부터 운송 저장 발전사업까지 아우르는 코오롱 H2 플랫폼을 구축한다고 밝히고, 장희구 사장은 국내외 수소산업 관련 투자자들을 대상으로 ‘코오롱 H2 플랫폼’의 산업적 가치와 의미를 설명하고 그룹의 역량을 집중하겠다고 발표했다.', 'query': '코오롱 수소 밸류체인 플랫폼 구축… 생산부터 발전까지 원스톱'}, {'answer': '한국동서발전은 다음달 31일까지 울산숲사랑운동과 함께 코로나19로 지친 시민들이 일상 속 걷기 운동을 통해 건강을 증진하고 자가용 대신 대중교통을 이용해 탄소중립 사회를 실현하기 위한 챌린지 탄소중립 건강걷기 누비GO 초록발자국 챌린지 를 진행한다고 1일 밝혔다.', 'query': '한국동서발전 울산시민과 함께하는 탄소중립 초록발자국 챌린지 진행'}, {'answer': '일동제약은 생산본부장으로 강덕원 부사장을 영입했다고 지난 1일 밝혔으며 강덕원 본부장은 서울대학교와 서던캘리포니아대학교 USC 를 졸업하고 동대학원에서 생화학 및 분자생물학 석사학위를 취득했으며 이후 글로벌 GMP 근무경력과 전문지식을 바탕으로 생산과 품질체계를 더욱 강화하고 cGMP 기준의 생산시설 구축도 추진하게 될 것이라고 말했다.', 'query': '일동제약 생산본부장에 강덕원 부사장 영입'}]], 'distances': [[0.2234136164188385, 1.0286645889282227, 1.167593240737915]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents

### langchain을 이용한 cromaDB 

In [22]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import CSVLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

loader = CSVLoader('../data/naver-news-summarization-ko/test_10row.csv',  encoding='cp949')
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma.from_documents(docs, embedding_function)

query = "코오롱 수소 밸류체인 플랫폼"
docs = db.similarity_search(query)

print(documents)
print('-' * 80)
print(docs)
print('-' * 80)
print(docs[0].page_content)

[Document(metadata={'source': '../data/naver-news-summarization-ko/test_10row.csv', 'row': 0}, page_content='date: 2022.7.4 8:47\ncategory: economy\npress: 머니투데이\ntitle: 아이트로닉스 차량용 복합기능형 졸음 방지 단말기 특허 출원\ndocument: 아이엘사이언스의 자회사 아이트로닉스는 차량용 복합기능형 졸음 방지 단말기 특허를 출원했다고 4일 밝혔다. 신규 특허는 자동차 주행 중 운전자의 졸음운전을 방지하는 상태 검출 기술에 관한 것이다. 해당 단말기는 가시광선 및 근적외선 광원을 조사하는 광원 모듈 운전자의 얼굴 영상을 촬영하는 가시광선 및 근적외선 카메라 차량 실내의 이산화탄소 농도를 측정하는 이산화탄소 센서로 구성됐다. 단말기는 광원에 반응하는 운전자의 얼굴 촬영 영상을 기반으로 심박 데이터와 눈의 깜빡임 횟수 눈을 감은 시간 등을 측정한다. 여기에 차내 졸음을 유발하는 이산화탄소 농도까지 종합적으로 분석해 운전자의 졸음 상태를 판단하고 결과값에 따라 경보 신호를 송출하도록 설계됐다. 아이트로닉스는 이번 특허기술을 차세대 지능형 교통체계 C ITS 시스템 설비에 적용할 예정이다. 회사 관계자는 이번 특허는 대표적인 차량 내적 사고 요인인 졸음운전 방지 차원에서 당사의 혁신 기술력을 집약해 정확도를 높이는 데 집중했다 며 완전 자율주행 단계에 이르기 전까지 지속될 운전자 안전사고 예방에 있어 해당 기술의 가시적인 성과를 기대하고 있다 고 말했다.\nlink: https://n.news.naver.com/mnews/article/008/0004766307?sid=101\nsummary: 아이일, 아이트로닉스는 차량용 복합기능형 졸음 방지 단말기 특허를 출원했다고 4일 밝혔으며 신규 특허는 자동차 주행 중 운전자의 졸음운전을 방지하는 상태 검출 기술에 관한 것으로, 해당 단말기는 가시광선 및 근적외선 광원을 조사하는 광원 모듈 

### FAISS 

In [24]:
from langchain_community.vectorstores import FAISS

# load the document and split it into chunks
loader = CSVLoader('../data/naver-news-summarization-ko/test_10row.csv',  encoding='cp949')
documents = loader.load()

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = FAISS.from_documents(docs, embedding_function)

# query it
query = "코오롱 수소 밸류체인 플랫폼"
docs = db.similarity_search(query)

# print results
print(docs[0].page_content)

date: 2022.7.6 16:27
category: economy
press: 데일리안
title: 코오롱 수소 밸류체인 플랫폼 구축… 생산부터 발전까지 원스톱
document: ‘코오롱 H2 플랫폼’ 밸류체인 플랫폼 발표 계열사간 역량 결집과 동시에 대외 파트너십 구축 6일 동대문 DDP에서 열린 코리아 H2서밋 인베스터데이에 참석한 코오롱인더스트리 장희구 사장 왼쪽에서 네번째 ⓒ코오롱그룹 데일리안 조인영 기자 코오롱그룹이 수소산업의 밸류체인 전반을 고도화하는 플랫폼 구축에 나선다. 코오롱그룹은 6일 동대문 디자인플라자에서 열린 ‘코리아 H2 비즈니스 서밋 2022 인베스터데이’에 참가해 코오롱인더스트리와 코오롱글로벌 코오롱글로텍 코오롱플라스틱 등 그룹이 보유한 첨단기술을 바탕으로 수소 생산부터 운송 저장 발전사업까지 아우르는 코오롱 H2 플랫폼을 구축한다고 밝혔다. 코오롱인더스트리 장희구 사장은 국내외 수소산업 관련 투자자들을 대상으로 ‘코오롱 H2 플랫폼’의 산업적 가치와 의미를 설명하고 그룹의 역량을 집중하겠다고 발표했다. 최근 세계적인 탄소중립 기조에 발맞춰 수소로 대표되는 청정에너지 사업을 그룹의 지속 가능한 미래를 만들 수 있는 핵심 사업으로 성장시키겠다는 전략이다. 코오롱그룹은 수소연료전지를 중심으로 전개해 온 사업 분야를 확대 재편해 청정수소 생산부터 운송과 저장 이를 이용한 전력생산까지 원스톱 서비스가 가능한 플랫폼을 마련해 나간다. 수소 생산분야는 국내 풍력발전 시장을 선도하고 있는 코오롱글로벌과 협업해 진행한다. 풍력발전은 특성상 야간 및 유휴전력이 발생하게 되는데 이를 활용해 물을 전기 분해하는 수전해방식으로 청정수소를 생산할 예정이다. 경주와 태백 풍력단지를 비롯해 현재 추진 중인 완도 해상풍력단지에 이르기까지 수소생산을 위한 중요한 기반 시설로 활용할 계획이다. 코오롱인더스트리의 수분제어장치 및 전해질 분리막 기술 PEM MEA 이 국내에서 선두를 달리고 있고 상용화도 가능해 사업화에 속도를 낼 수 있을 것으로 보고 있다. 코오롱